### Custom Name Entity Check Func

This notebook uses for finding name entity in plain text data. Note: This notebook should run after word tokenize notebook because word_list must give this notebook. This notebook process could takes more time.

In [2]:
import pandas as pd
import numpy as np
from itertools import islice
import re

In [ ]:
#df_word = pd.read_csv("..../Word_Tokenize.csv")

In [ ]:
## main data
#word_list = list(set(df_word["word"]))

In [108]:
# test data
word_list = ["ramiro", "Thomas", "Peter","Arthur", "Kirkland", "Sizi","Ertugrul", "Gregor", "$ali", "20str", "evet","neden", "archie", "gaby", "dani"]  # word list acquires with word tokenize with set

In [109]:
with open("/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Pyspark/ReDe/xaa.tr", "r",encoding="latin1") as file:  # sample data
    text = file.read()

In [110]:
digit_and_not_alnum = []
string_word = []
for i in word_list:
#for i in df_word.sentence:
    i = str(i)
    if not(i[0].isalnum()) or i[0].isdigit():
        digit_and_not_alnum.append(i)
    else:
        string_word.append(i)

In [175]:
# # 1-Way Slow But More Accurancy
# special_name_list = []  
# for j in string_word: 
#     # var1_list = re.findall(f"(?:\s|^){j}", text, flags=re.IGNORECASE)  # begin with
#     var1_list = re.findall(f"(?:\s|^){j}(?:\s|$)", text, flags=re.IGNORECASE)
#     var2_list = []
#     for k in var1_list:
#         k = re.findall(r'\w+', k)
#         var2_list.append(k[0])
#     upper_list = []
#     lower_list = []
#     for m in var2_list:
#         if m[0] == m[0].upper():
#             upper_list.append(m)
#         else:
#             lower_list.append(m)
#     if round((len(upper_list))/(len(upper_list)+len(lower_list)+0.1)*100,5) >= 98:
#         special_name_list.append(j.capitalize())
#     else:
#         pass
# special_name_list

['Ramiro', 'Thomas', 'Peter', 'Arthur', 'Kirkland']

In [191]:
# 3-Way Fast But Less Accurancy  
# The function takes a hundred samples for the words given in the word list and checks their capitalization and decides whether it is a name entity or not.   
special_name_list = []  
for j in string_word:
    var1_list = []
    limit = 100  # sample number
    # for x in islice(re.finditer(f"(?:\s|^){j}", text, flags=re.IGNORECASE), limit):
    for x in islice(re.finditer(f"(?:\s|^){j}(?:\s|$)", text, flags=re.IGNORECASE), limit):
        var1_list.append(x.group())
    upper_list = []
    lower_list = []
    for m in var1_list:
        if m[0] == m[0].upper():
            upper_list.append(m)
        else:
            lower_list.append(m)
    if round((len(upper_list))/(len(upper_list)+len(lower_list)+0.1)*100,5) >= 99:
        special_name_list.append(j.capitalize())
    else:
        pass
special_name_list

['Thomas', 'Peter', 'Arthur', 'Kirkland', 'Sizi', 'Evet', 'Neden']

In [ ]:
df_special_name = pd.DataFrame(special_name_list)
df_special_name

In [ ]:
df_special_name.to_excel("Special_Name.xlsx", sheet_name="Special_Name", index=False)

#### Multi Processing

In [26]:
import pandas as pd
import numpy as np
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
import re

In [27]:
help(multiprocessing)

Help on package multiprocessing:

NAME
    multiprocessing

MODULE REFERENCE
    https://docs.python.org/3.9/library/multiprocessing
    
    The following documentation is automatically generated from the Python
    source files.  It may be incomplete, incorrect or include features that
    are considered implementation detail and may vary between Python
    implementations.  When in doubt, consult the module reference at the
    location listed above.

DESCRIPTION
    # Package analogous to 'threading.py' but using processes
    #
    # multiprocessing/__init__.py
    #
    # This package is intended to duplicate the functionality (and much of
    # the API) of threading.py but uses processes instead of threads.  A
    # subpackage 'multiprocessing.dummy' has the same API but is a simple
    # wrapper for 'threading'.
    #
    # Copyright (c) 2006-2008, R Oudkerk
    # Licensed to PSF under a Contributor Agreement.
    #

PACKAGE CONTENTS
    connection
    context
    dummy (packag

In [28]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

Number of CPU cores: 16


In [ ]:
folder_name = "Turkish"  # English, Turkish, German, French, Spanish, Arabic, Portuguese, Dutch, Italian
 
if folder_name == "English":
    text_name = "en"
elif folder_name == "Turkish":
    text_name = "tr"
elif folder_name == "German":
    text_name = "de"
elif folder_name == "French":
    text_name = "fr"
elif folder_name == "Spanish":
    text_name = "es"
elif folder_name == "Arabic":
    text_name = "ar"
elif folder_name == "Portuguese":
    text_name = "pt"
elif folder_name == "Dutch":
    text_name = "nl"
elif folder_name == "Italian":
    text_name = "it"
else:
    pass
#custom_alp = tr  # en, tr, de, fr, es, ar, pt, nl, it
print(f"{folder_name}\n{text_name}\n") 

In [ ]:
df_word = pd.read_csv("..../Not_Apply_Lower_Word_Merge_Result.csv")  # from (Word/2-Merge Word Sentence N Gram Rev2.ipynb) notebook data result
df_word

In [ ]:
# main data
word_list = list(set(df_word["word"]))

In [ ]:
pre_process_plain_text_path = "/home/ubuntu/Workspace"
part_num = 1

In [ ]:
with open(f"{pre_process_plain_text_path}/{text_name}{part_num}.txt", "r", encoding="utf-8") as file:  # sample data
    text = file.read()

In [ ]:
special_name_list = []

In [ ]:
# Not: Pool uzun sürecek olan işlemler için Process den daha iyi
# The function takes a hundred samples for the words given in the word list and checks their capitalization and decides whether it is a name entity or not.
# import os
# from multiprocessing import Process, Manager
manager = multiprocessing.Manager()
special_name_list = manager.list()

digit_and_not_alnum = []
string_word = []
for i in word_list:
    i = str(i)
    if not(i[0].isalnum()) or i[0].isdigit():
        digit_and_not_alnum.append(i)
    else:
        string_word.append(i)

def find_special_name(name_ent):
    var_list = []
    limit = 100  # sample number
    #for x in islice(re.finditer(f"(?:\s|^){name_ent}", text, flags=re.IGNORECASE), limit): # begin with
    for x in islice(re.finditer(f"(?:\s|^){name_ent}(?:\s|$)", text, flags=re.IGNORECASE), limit): # exact match
        var_list.append(x.group())
    upper_list = []
    lower_list = []
    for m in var_list:
        if m[0] == m[0].upper():
            upper_list.append(m)
        else:
            lower_list.append(m)
    if round((len(upper_list))/(len(upper_list)+len(lower_list)+0.01)*100,5) >= 99:
        special_name_list.append(name_ent.capitalize())
    else:
        special_name_list.append(f"not special name {name_ent}")

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs-2) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(find_special_name, string_word) # string_word liste

In [ ]:
list_special_name = list(special_name_list)
df_special_name = pd.DataFrame(list_special_name)
df_special_name.rename(columns={0:"spec_name"}, inplace=True)
df_special_name

In [ ]:
df_special_name.to_excel(f"Special_Name{part_num}.xlsx", index=False)